#Imports

In [ ]:
import numpy as np
import math
from scipy.optimize import linprog
import random
import matplotlib.pyplot as plt

In [ ]:
n_arms = 3
#n_arms = 2
#hospitals = 3
hospitals = 5

#Update UCB

In [ ]:
def new_ucb(r, N, t, c):
  ucb = []
  lcb = []
  for i in range(n_arms):
    if N[i] == 0:
      ucb.append(1)
      lcb.append(0)
    else:
      ucb.append(np.min([r[i]/N[i] + c*math.sqrt(math.log(t)/N[i]), 1]))
      lcb.append(np.max([r[i]/N[i] - c*math.sqrt(math.log(t)/N[i]), 0]))
      #ucb.append(np.min([r[i]/N[i] + 0.8*math.sqrt(math.log(t)/N[i]), 1]))
      #lcb.append(np.max([r[i]/N[i] - 0.8*math.sqrt(math.log(t)/N[i]), 0]))
  return ucb, lcb

#Train Function

In [ ]:
def train_random(pi, responses, rewards, timestep):
  succ = []
  estimated_reward = []
  success = []
  times = []
  for i in range(hospitals):
    estimated_reward.append([])
    success.append([])
    succ.append(0)
    times.append(0)
  regret = []
  r = []
  N = []
  index = []
  reg = []
  for i in range(hospitals):
    regret.append([])
  #for i in range(n_arms):
  #  r.append(0)
  #for i in range(n_arms):
  #  N.append(0)
  for i in range(hospitals):
    l = []
    lr = []
    ln = []
    for j in range(n_arms):
      l.append(0)
      lr.append(0)
      ln.append(0)
    index.append(l)
    r.append(lr)
    N.append(ln)
    reg.append(0)
  #############################
  for time in range(timestep):
    clus = []
    for i in range(hospitals):
      clus.append(np.random.choice([1, 0], p = [pi[i], 1-pi[i]]))
    for i in range(hospitals):
      if clus[i] == 1:
        arm = np.random.choice([0, 1, 2])
        #arm = np.random.choice([0, 1])
        times[i] += 1
        if arm == np.argmax(rewards[i]):
          succ[i] += 1
        success[i].append(succ[i]/times[i])
        N[i][arm] += 1
        prob = rewards[i][arm]
        #print(index)
        r[i][arm] += responses[i][arm][index[i][arm]]
        index[i][arm] += 1
        reg[i] += np.max(rewards[i]) - prob
        regret[i].append(reg[i])
        arm_now = np.argmax(rewards[i])
        if N[i][arm_now] != 0:
          estimated_reward[i].append(r[i][arm_now]/N[i][arm_now])
        else:
          estimated_reward[i].append(0)
  return regret, estimated_reward, success

In [ ]:
def train_all(pi, responses, rewards, timesteps, bool_causal):
  prob_explore = 0.5
  succ = []
  estimated_reward = []
  success = []
  times = []
  for i in range(hospitals):
    estimated_reward.append([])
    success.append([])
    succ.append(0)
    times.append(0)
  ucb = []
  lcb = []
  ############
  regret = []
  r = []
  N = []
  Ni = []
  ri = []
  t = 0
  index = []
  reg = []
  #########################
  for j in range(n_arms):
    ucb.append(1)
    lcb.append(0)
  for i in range(hospitals):
    regret.append([])
  for i in range(n_arms):
    r.append(0)
  for i in range(n_arms):
    N.append(0)
  for i in range(hospitals):
    l = []
    lr = []
    ln = []
    for j in range(n_arms):
      l.append(0)
      lr.append(0)
      ln.append(0)
    index.append(l)
    ri.append(lr)
    Ni.append(ln)
    reg.append(0)
  ######################
  for time in range(timesteps):
    clus = []
    for i in range(hospitals):
      clus.append(np.random.choice([1, 0], p = [pi[i], 1-pi[i]]))
    if t%100 == 0:
      ucb, lcb = new_ucb(r, N, t, 0.8)
    for i in range(hospitals):
      if clus[i] == 1:
        arm = np.random.choice(np.where(ucb == np.max(ucb))[0])
        ######HERE
        if time < 100:
          flag = np.random.choice([1, 0], p = [prob_explore, 1- prob_explore])
          if flag == 1:
            ##CHANGE WITH ARMS
            arm = np.random.choice([0, 1, 2])
            #arm = np.random.choice([0, 1])
        times[i] += 1
        if arm == np.argmax(rewards[i]):
          succ[i] += 1
        success[i].append(succ[i]/times[i])
        N[arm] += 1
        Ni[i][arm] += 1
        t += 1
        prob = rewards[i][arm]
        #print(index)
        r[arm] += responses[i][arm][index[i][arm]]
        ri[i][arm] += responses[i][arm][index[i][arm]]
        index[i][arm] += 1
        reg[i] += np.max(rewards[i]) - prob
        regret[i].append(reg[i])
        arm_now = np.argmax(rewards[i])
        if Ni[i][arm_now] != 0:
          estimated_reward[i].append(ri[i][arm_now]/Ni[i][arm_now])
        else:
          estimated_reward[i].append(0)
###########################################
###########################################
    if time %100 == 0 and time != 0:
      prob_explore /= 1.1
  return regret, estimated_reward, success